In [8]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys

caffe_root = 'caffe/'
sys.path.insert(0, caffe_root + 'python')
import caffe

In [49]:
# some magic with DF
inp_data = pd.DataFrame.from_csv('faces.csv')
idx = np.array([i for i in xrange(inp_data.shape[0])])
inp_data['idx'] = idx
inp_data['frame'] = inp_data.index
data = inp_data.set_index('idx')
data['age'] = np.zeros(data.shape[0])
data['gender'] = np.zeros(data.shape[0])
data = data.head(5) # ибо очень долго считает, потом надо на всем норм попредиктить

In [50]:
def make_im_name(frame, person_id):
    return "frame%dperson%d.jpg" % (frame, person_id)

In [51]:
age_net_pretrained='./caffe_models/dex_imdb_wiki.caffemodel'
age_net_model_file='./caffe_models/age.prototxt'
age_net = caffe.Classifier(age_net_model_file, age_net_pretrained,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

gender_net_pretrained='./caffe_models/gender.caffemodel'
gender_net_model_file='./caffe_models/gender.prototxt'
gender_net = caffe.Classifier(gender_net_model_file, gender_net_pretrained,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [53]:
age_list= [x for x in range(101)]
gender_list = ['Female', 'Male']

for i in data.index:
    input_image = caffe.io.load_image('./faces/' + make_im_name(data['frame'][i], data['person_id'][i]))
    data['age'][i] = age_net.predict([input_image])[0].argmax()
    data['gender'][i] = gender_list[gender_net.predict([input_image])[0].argmax()]

In [54]:
data

,person_id,x,y,w,h,frame,age,gender
idx,,,,,,,,
0,1,581,35,117,117,0,39,Male
1,2,123,101,118,118,0,53,Male
2,1,582,34,118,118,1,37,Male
3,2,122,101,119,119,1,48,Male
4,1,582,31,123,123,2,37,Male


In [55]:
data.to_csv('recognized.csv')